### Imports

In [5]:
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

### Connect to the Neo4j database and instantiate an LLM Transformer

In [8]:
openai_apikey = ""
url = "bolt://localhost:7687"
username = "neo4j"
password = ""
graph = Neo4jGraph(url=url, username=username, password=password)

llm = ChatOpenAI(temperature=0, model_name="gpt-4-0125-preview", openai_api_key=openai_apikey)
llm_transformer = LLMGraphTransformer(llm=llm)

### Provide Email Text
This is a naive approach that takes in one email and converts it to entities and relationships in the Neo4j graph. Ideally we want to read a set of emails and perform this operation for all of them.

In [12]:
text = """
Team,

Thank you for a productive meeting today. I appreciate everyone's insights and contributions. Please proceed with the actions discussed, and let's aim to have the draft report ready for review by next Wednesday. Feel free to reach out if you encounter any issues.

Best regards,
Emily
"""

### Build the Knowledge Graph (KG) 

The code below is for transforming text into a graph structure. Here, you are creating a list of Document objects. Each Document object is initialized with some content (text). The variable text should contain the textual data you want to process. The llm_transformer (which stands for Language Model Transformer) is used to convert the list of Document objects into a format suitable for graph representation. 
We are concerned about:
* Entity Recognition: Identifying entities within the text.
* Relationship Extraction: Determining the relationships between identified entities.
* Graph Construction: Constructing a graph where nodes represent entities and edges represent relationships between these entities.

In [13]:
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

graph.add_graph_documents(graph_documents)

Nodes:[Node(id='Productive Meeting', type='Event'), Node(id='Draft Report', type='Document'), Node(id='Emily', type='Person')]
Relationships:[Relationship(source=Node(id='Emily', type='Person'), target=Node(id='Productive Meeting', type='Event'), type='THANKED_FOR'), Relationship(source=Node(id='Draft Report', type='Document'), target=Node(id='Productive Meeting', type='Event'), type='DISCUSSED_AT'), Relationship(source=Node(id='Draft Report', type='Document'), target=Node(id='Emily', type='Person'), type='REVIEW_BY')]
